# Asymmetric Quantum Well

In [1]:
from sympy import *
from sympy.physics.mechanics import *
from sympy import sin, cos
import sympy as sp
init_printing()

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

import scipy as spy
import scipy.special as special
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import curve_fit

large_width = 180
np.set_printoptions(precision=3,linewidth=large_width,suppress=False) 


/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
L,hbar,mass,a,x,m,n = symbols('L,hbar,mass,a,x,m,n', real = True) 
#Set up params as symbols for differentiation


#Use dynamic symbols from sympy to create differential terms
y1 = sin(pi*m*x/L)
y2 = sin(pi*n*x/L)
ddx = diff(y2, x, 2)

def Int(L,hbar,mass,pi,m,n,x):
    return -hbar**2/(mass*L) * y1*ddx

def Int2(L,pi,a,m,n,x):
    return (2*a/L**2) * x*y1*y2

int1 = sp.integrate(Int(L,hbar,mass,pi,m,n,x),(x,0,L))
int2 = sp.integrate(Int2(L,pi,a,m,n,x), (x,0,L))

int1 = int1.subs([(sin(n*pi),0), (cos(n*pi),(-1)**n), (sin(m*pi),0), (cos(m*pi),(-1)**m)])
int2 = int2.subs([(sin(n*pi),0), (cos(n*pi),(-1)**n), (sin(m*pi),0), (cos(m*pi),(-1)**m)])

int1 = sp.lambdify([L,hbar,mass,pi,m,n], int1, "numpy")
int2 = sp.lambdify([L,pi,a,m,n], int2, "numpy")

L = 5e-10 #meters or 5 Angstroms
hbar = 1.0545e-34 # Joule sec
mass = 9.11e-31 # kg
pi = pi
a = 1.6022e-18 # Joules
Nvals = [11,101]
Mvals = [11,101]
for M in Mvals:
    for N in Nvals:
        if N != M:
            continue
        Hml = np.empty((M-1,N-1))
        for m in range (1,M):
            for n in range(1,N):
                if m == n:
                    continue
                elif m % 2 == 0 and n % 2 == 0:
                    continue
                elif n % 2 != 0 and m % 2 != 0:
                    continue
                Hml[m-1,n-1] = int2(L,sp.pi,a,m,n) + int1(L,hbar,mass,sp.pi,m,n)


        for m in range (1,M):
            for n in range(1,N):
                if m != n:
                    continue
                Hml[m-1,n-1] = int2(L,sp.pi,a,m,n) + int1(L,hbar,mass,sp.pi,m,n)

        eps = 1e-100
        eps2 = 1e100
        Hml[np.abs(Hml) < eps] = 0   
        Hml[np.abs(Hml) > eps2] = 0  
        if N == M == 11:
            Hml_10 = Hml
        else:
            Hml_100 = Hml
print(Hml_10)
print(Hml_100)

        #Find equations of motion with Euler-Lagrange equations for z and theta coordinates

        #print(Integrand)

[[ 1.042e-18 -2.886e-19  0.000e+00 -2.309e-20  0.000e+00 -6.361e-21  0.000e+00 -2.618e-21  0.000e+00 -1.325e-21]
 [-2.886e-19  1.765e-18 -3.117e-19  0.000e+00 -2.945e-20  0.000e+00 -8.979e-21  0.000e+00 -3.943e-21  0.000e+00]
 [ 0.000e+00 -3.117e-19  2.970e-18 -3.180e-19  0.000e+00 -3.207e-20  0.000e+00 -1.030e-20  0.000e+00 -4.705e-21]
 [-2.309e-20  0.000e+00 -3.180e-19  4.656e-18 -3.207e-19  0.000e+00 -3.339e-20  0.000e+00 -1.107e-20  0.000e+00]
 [ 0.000e+00 -2.945e-20  0.000e+00 -3.207e-19  6.825e-18 -3.220e-19  0.000e+00 -3.415e-20  0.000e+00 -1.154e-20]
 [-6.361e-21  0.000e+00 -3.207e-20  0.000e+00 -3.220e-19  9.475e-18 -3.228e-19  0.000e+00 -3.463e-20  0.000e+00]
 [ 0.000e+00 -8.979e-21  0.000e+00 -3.339e-20  0.000e+00 -3.228e-19  1.261e-17 -3.232e-19  0.000e+00 -3.495e-20]
 [-2.618e-21  0.000e+00 -1.030e-20  0.000e+00 -3.415e-20  0.000e+00 -3.232e-19  1.622e-17 -3.236e-19  0.000e+00]
 [ 0.000e+00 -3.943e-21  0.000e+00 -1.107e-20  0.000e+00 -3.463e-20  0.000e+00 -3.236e-19  2.032

In [3]:
Nvals = [11,101]
Mvals = [11,101]
for M in Mvals:
    for N in Nvals:
        if N != M:
            continue
        H = np.empty((M-1,N-1))

        def offdiagonal(a,m,n):
            return -8*a*m*n / ((np.pi*(m**2-n**2))**2)

        def diagonal(hbar,n,L,mass,a):
            return (hbar*n*np.pi/L)**2/(2*mass) + a/2

        for m in range (1,M):
            for n in range(1,N):
                if m == n:
                    continue
                elif m % 2 == 0 and n % 2 == 0:
                    continue
                elif n % 2 != 0 and m % 2 != 0:
                    continue
                H[m-1,n-1] = offdiagonal(a,m,n)

        for m in range (1,M):
            for n in range(1,N):
                if m != n:
                    continue
                H[m-1,n-1] = diagonal(hbar,n,L,mass,a)
        H[np.abs(H) < eps] = 0   
        H[np.abs(H) > eps2] = 0  
        if N == M == 11:
            H_10 = H
        else:
            H_100 = H
print(H_10)
print(H_100)

[[ 1.042e-18 -2.886e-19  0.000e+00 -2.309e-20  0.000e+00 -6.361e-21  0.000e+00 -2.618e-21  0.000e+00 -1.325e-21]
 [-2.886e-19  1.765e-18 -3.117e-19  0.000e+00 -2.945e-20  0.000e+00 -8.979e-21  0.000e+00 -3.943e-21  0.000e+00]
 [ 0.000e+00 -3.117e-19  2.970e-18 -3.180e-19  0.000e+00 -3.207e-20  0.000e+00 -1.030e-20  0.000e+00 -4.705e-21]
 [-2.309e-20  0.000e+00 -3.180e-19  4.656e-18 -3.207e-19  0.000e+00 -3.339e-20  0.000e+00 -1.107e-20  0.000e+00]
 [ 0.000e+00 -2.945e-20  0.000e+00 -3.207e-19  6.825e-18 -3.220e-19  0.000e+00 -3.415e-20  0.000e+00 -1.154e-20]
 [-6.361e-21  0.000e+00 -3.207e-20  0.000e+00 -3.220e-19  9.475e-18 -3.228e-19  0.000e+00 -3.463e-20  0.000e+00]
 [ 0.000e+00 -8.979e-21  0.000e+00 -3.339e-20  0.000e+00 -3.228e-19  1.261e-17 -3.232e-19  0.000e+00 -3.495e-20]
 [-2.618e-21  0.000e+00 -1.030e-20  0.000e+00 -3.415e-20  0.000e+00 -3.232e-19  1.622e-17 -3.236e-19  0.000e+00]
 [ 0.000e+00 -3.943e-21  0.000e+00 -1.107e-20  0.000e+00 -3.463e-20  0.000e+00 -3.236e-19  2.032

In [4]:
#For M = N = 11
PythonEigvals = np.linalg.eigvalsh(H_10*6.242e+18)
AnalyticEigvals = np.linalg.eigvalsh(Hml_10*6.242e+18)

PythonEigvals = np.round(PythonEigvals,3)
AnalyticEigvals = np.round(AnalyticEigvals,3)
print(PythonEigvals,'eV')
print(AnalyticEigvals,'eV')

[  5.836  11.181  18.662  29.142  42.651  59.179  78.721 101.274 126.837 155.537] eV
[  5.836  11.181  18.662  29.142  42.651  59.179  78.721 101.274 126.837 155.537] eV


In [5]:
#M = N = 101
Py = np.linalg.eigvalsh(H_100*6.242e+18)
Anlyt = np.linalg.eigvalsh(Hml_100*6.242e+18)

Py = np.round(Py,6)
Anlyt = np.round(Anlyt,6)
print(Py[0:10],'eV')
print(Anlyt[0:10],'eV')

[  5.836  11.181  18.662  29.142  42.651  59.179  78.721 101.273 126.836 155.407] eV
[  5.836  11.181  18.662  29.142  42.651  59.179  78.721 101.273 126.836 155.407] eV


In [6]:
print(np.char.rjust(str(Py[0:10] - PythonEigvals),width = 1500))
print(np.char.rjust(str(Anlyt[0:10] - AnalyticEigvals),width = 1500))